In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
columns_names=['class','age', 'sex', 'steroid', 'antivirals', 'fatigue', 'malaise', 'anorexia',
       'liver_big', 'liver_firm', 'spleen_palpable', 'spiders', 'ascites',
       'varices', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin', 'protime',
       'histology']

In [52]:
df=pd.read_csv('hepatitis.data')
df.columns=columns_names

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   class            154 non-null    int64 
 1   age              154 non-null    int64 
 2   sex              154 non-null    int64 
 3   steroid          154 non-null    object
 4   antivirals       154 non-null    int64 
 5   fatigue          154 non-null    object
 6   malaise          154 non-null    object
 7   anorexia         154 non-null    object
 8   liver_big        154 non-null    object
 9   liver_firm       154 non-null    object
 10  spleen_palpable  154 non-null    object
 11  spiders          154 non-null    object
 12  ascites          154 non-null    object
 13  varices          154 non-null    object
 14  bilirubin        154 non-null    object
 15  alk_phosphate    154 non-null    object
 16  sgot             154 non-null    object
 17  albumin          154 non-null    ob

In [54]:
df.head()

,class,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology
0,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.90,135,42,3.5,?,1
1,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.70,96,32,4.0,?,1
2,2,31,1,?,1,2,2,2,2,2,2,2,2,2,0.70,46,52,4.0,80,1
3,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.00,?,200,4.0,?,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,0.90,95,28,4.0,75,1


In [55]:
from sklearn.impute import KNNImputer
df.replace('?',np.nan,inplace=True)
X=df.iloc[:,1:20].values
Y=df.iloc[:,0].values

In [56]:
imputer=KNNImputer(n_neighbors=3)
X=imputer.fit_transform(X)

In [57]:
np.unique(X[:,13])

array([0.3       , 0.4       , 0.5       , 0.6       , 0.7       ,
       0.8       , 0.9       , 1.        , 1.03333333, 1.1       ,
       1.2       , 1.3       , 1.4       , 1.5       , 1.56666667,
       1.6       , 1.7       , 1.8       , 1.9       , 1.93333333,
       2.        , 2.16666667, 2.2       , 2.3       , 2.4       ,
       2.5       , 2.8       , 2.9       , 3.        , 3.2       ,
       3.5       , 3.9       , 4.1       , 4.2       , 4.6       ,
       4.8       , 7.6       , 8.        ])

مرزبندی واضحی برای این ستون دیده نشد 
bilirubin

In [58]:
from sklearn.preprocessing import OneHotEncoder
coding=OneHotEncoder()
cols=['sex','steroid','antivirals','fatigue','malaise','anorexia','liver_big','liver_firm','spleen_palpable','spiders','ascites','varices','histology']

In [59]:
for col in cols:
    coded=coding.fit_transform(df[col].to_numpy().reshape(-1,1)).toarray()
    X=np.concatenate((X,coded),axis=1)
    index=df.columns.get_loc(col)+1
    X=np.delete(X,index,1)

    

In [60]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(X, Y , test_size=0.10 , random_state=85 )

In [74]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(k_neighbors=1,random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, Y_train.ravel())

# SGD

In [175]:
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(loss='squared_hinge',alpha=0.01,penalty='l1')
clf.fit(X_train_res,y_train_res)

SGDClassifier(alpha=0.01, loss='squared_hinge', penalty='l1')

In [176]:
y_predict=clf.predict(x_test)


In [177]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test,y_predict))


[[ 3  1]
 [ 0 12]]


In [178]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,X_train_res,y_train_res,cv=3)


array([0.75675676, 0.8630137 , 0.78082192])